In [1]:
using LinearAlgebra
using Jacobi
using Plots
using Test

In [2]:
function GetQuadrature2D(Q)
    
    alpha = 0.0
    beta = 0.0
    q = zgj(Q, alpha, beta)
    w = wgj(q, alpha, beta)
    
    w2 = zeros(Q*Q)
    qx = zeros(Q*Q)
    qy = zeros(Q*Q)
    for i=1:Q
        for j=1:Q
            k = (i-1)*Q +j
            qx[k] = q[j]
            qy[k] = q[i]
            w2[k] = w[j]*w[i]
        end
    end
    return w2, qx, qy
end

GetQuadrature2D (generic function with 1 method)

In [3]:
function BilinearMap(coord_E, xhat, yhat)
    m = length(xhat)
    P = 0.25 * [(1 .- xhat).*(1 .- yhat) (1 .+ xhat).*(1 .- yhat) (1 .- xhat).*(1 .+ yhat) (1 .+ xhat).*(1 .+ yhat)]
    P_E = P * coord_E
    # x=F_E(xhat)
    X = P_E
    x = X[:, 1]
    y = X[:, 2]
    # gradient of P, 1st row = dP/dxhat, 2nd row=dP/dyhat
    # GradP_(2m x 4)
    GradP = 0.25 * [-(1 .- yhat) (1 .- yhat) -(1 .+ yhat) (1 .+ yhat);
                    -(1 .- xhat) -(1 .+ xhat) (1 .- xhat) (1 .+ xhat)]

    # JT= [[dx/dxhat, dy/dxhat],
    #      [dx/dyhat, dy/dyhat]] (2m x 2)
    JT = GradP * coord_E
    dxdxhat = JT[1:m,1]
    dydxhat = JT[1:m,2]
    dxdyhat = JT[m+1:2*m,1]
    dydyhat = JT[m+1:2*m,2]
    J_E = dxdxhat .* dydyhat .- dydxhat .* dxdyhat
    
    DF_E = [dxdxhat dxdyhat;dydxhat dydyhat]
    
    return x, y, DF_E, J_E
end

BilinearMap (generic function with 1 method)

In [4]:
@testset "uniformgrid" begin
    # integrate over [0,2]x[0,1]
    Q = 5
    coord_E = [0. 0.;2. 0.;0. 1.;2. 1.]
    w2, qx, qy = GetQuadrature2D(Q)
    x, y, DF_E, J_E = BilinearMap(coord_E,qx, qy)
    @test isapprox((J_E .* w2)' * (x .* y.^2), 2/3, atol=1-13)
    @test isapprox((J_E .* w2)' * (cos.(x) .* cos.(y)), sin(1)*sin(2), atol=1-6)
    
end

Test Summary: | Pass  Total
uniformgrid   |    2      2


Test.DefaultTestSet("uniformgrid", Any[], 2, false, false)

In [5]:
@testset "nonuniformgrid" begin
    
    # integrate over quad
    Q = 3
    coord_E = [0. 0.;48. 64.;-60. 40.;0. 100.]
    w2, qx, qy = GetQuadrature2D(Q)
    x, y, DF_E, J_E = BilinearMap(coord_E,qx, qy)
    @test isapprox((J_E .* w2)' * (ones(Q*Q)), 5400, atol=1-13)
    
end

Test Summary:  | Pass  Total
nonuniformgrid |    1      1


Test.DefaultTestSet("nonuniformgrid", Any[], 1, false, false)

In [6]:
# integrate Legendre Polynomial over quad
Q = 5
coord_E = [0. 0.;2. 0.;0. 1.;2. 1.]
w2, qx, qy = GetQuadrature2D(Q)
x, y, DF_E, J_E = BilinearMap(coord_E,qx, qy)
leg22 = legendre.(qx, 2) .* legendre.(qy, 2)
leg12 = legendre.(qx, 1) .* legendre.(qy, 2)
(J_E .* w2)' * (leg22 .* leg12)

0.0